## **1. Pendahuluan**
Sistem rekomendasi adalah teknologi yang digunakan untuk memberikan rekomendasi atau saran kepada pengguna berdasarkan preferensi mereka. Dalam proyek ini, kami akan menggunakan dataset MovieLens dan membangun sistem rekomendasi menggunakan algoritma Collaborative Filtering dengan metode Matrix Factorization, yaitu Singular Value Decomposition (SVD).

## **2. Langkah-langkah**
### **2.1. Persiapan Awal**
Impor library yang diperlukan, seperti pandas, numpy, dan scikit-learn. Menginstal library surprise yang digunakan untuk algoritma rekomendasi.

In [6]:
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np

### **2.2. Unduh dan Muat Dataset**
Mengunduh dataset MovieLens dan memuat dataset ke dalam DataFrame menggunakan library pandas. Kita akan menggunakan dataset MovieLens 100K.

In [7]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip


--2023-06-06 10:56:24--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  11.3MB/s    in 0.4s    

2023-06-06 10:56:25 (11.3 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
# Muat data pengguna
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')

# Muat data peringkat
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# Muat data film
movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=movies_cols, usecols=range(5), encoding='latin-1')

### **2.3. Eksplorasi Dataset**
Melakukan eksplorasi dataset untuk memahami struktur dan informasi yang terkandung di dalamnya. Menggunakan perintah seperti head(), info(), dan describe() untuk melihat beberapa baris awal, informasi kolom, dan statistik deskriptif.

In [12]:
print(users.head())

   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


In [13]:
print(ratings.head())

   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596


In [14]:
print(movies.head())

   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            imdb_url  
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...  
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...  
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...  
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...  
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)  


In [15]:
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     943 non-null    int64 
 1   age         943 non-null    int64 
 2   sex         943 non-null    object
 3   occupation  943 non-null    object
 4   zip_code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB
None


In [16]:
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   user_id         100000 non-null  int64
 1   movie_id        100000 non-null  int64
 2   rating          100000 non-null  int64
 3   unix_timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
None


In [17]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   title               1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   imdb_url            1679 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 65.8+ KB
None


In [18]:
print(ratings.describe())

            user_id       movie_id         rating  unix_timestamp
count  100000.00000  100000.000000  100000.000000    1.000000e+05
mean      462.48475     425.530130       3.529860    8.835289e+08
std       266.61442     330.798356       1.125674    5.343856e+06
min         1.00000       1.000000       1.000000    8.747247e+08
25%       254.00000     175.000000       3.000000    8.794487e+08
50%       447.00000     322.000000       4.000000    8.828269e+08
75%       682.00000     631.000000       4.000000    8.882600e+08
max       943.00000    1682.000000       5.000000    8.932866e+08


### **2.4. Persiapan Data**
Melakukan pemrosesan data yang diperlukan, seperti menggabungkan DataFrame, menghapus kolom yang tidak diperlukan, dan mengubah format data jika diperlukan.

In [19]:
# Menggabungkan data pengguna, peringkat, dan film
data = pd.merge(pd.merge(ratings, users), movies)

# Hapus kolom yang tidak diperlukan
data = data.drop(['unix_timestamp', 'zip_code', 'release_date', 'video_release_date', 'imdb_url'], axis=1)

# Cetak jumlah unik pengguna dan film
num_users = data.user_id.nunique()
num_movies = data.movie_id.nunique()

print("Jumlah Pengguna:", num_users)
print("Jumlah Film:", num_movies)


Jumlah Pengguna: 943
Jumlah Film: 1682


### **2.5. Membangun Model**
Memilih model rekomendasi yang ingin digunakan, dalam hal ini memilih SVD (Singular Value Decomposition) dari library surprise.

In [21]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

In [27]:
# Membuat objek Reader untuk membaca dataset
reader = Reader(rating_scale=(1, 5))

In [28]:
# Memuat dataset MovieLens ke objek Dataset
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

In [29]:
# Inisialisasi model SVD
svd = SVD()

In [30]:
# Evaluasi model menggunakan cross-validation
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9382  0.9340  0.9302  0.9412  0.9325  0.9352  0.0040  
MAE (testset)     0.7388  0.7354  0.7344  0.7412  0.7362  0.7372  0.0025  
Fit time          1.45    1.23    1.20    1.20    1.21    1.26    0.10    
Test time         0.23    0.12    0.21    0.14    0.24    0.19    0.05    


{'test_rmse': array([0.93818387, 0.93395552, 0.93018382, 0.94122284, 0.93254614]),
 'test_mae': array([0.73882575, 0.73543344, 0.73437856, 0.74121173, 0.73623634]),
 'fit_time': (1.4546494483947754,
  1.2288904190063477,
  1.2017126083374023,
  1.2030389308929443,
  1.210170030593872),
 'test_time': (0.22869324684143066,
  0.12085676193237305,
  0.21160244941711426,
  0.14128684997558594,
  0.23996186256408691)}

### **2.6. Melatih Model dan Menguji Sistem**
Melatih model rekomendasi menggunakan seluruh data rating dan menguji sistem dengan memberikan rekomendasi kepada pengguna.

In [58]:
# Latih model dengan seluruh data rating
trainset = data.build_full_trainset()
svd.fit(trainset)

# Contoh memberikan rekomendasi untuk pengguna dengan ID 5
user_id = 5
user_ratings = ratings[ratings['user_id'] == user_id]
user_unseen_movies = movies[~movies['movie_id'].isin(user_ratings['movie_id'])]

# Buat prediksi rating untuk film yang belum ditonton pengguna
user_unseen_movies['predicted_rating'] = user_unseen_movies['movie_id'].apply(lambda x: svd.predict(user_id, x).est)

# Urutkan film berdasarkan prediksi rating
user_recommendations = user_unseen_movies.sort_values(by='predicted_rating', ascending=False).head(5)


<ipython-input-58-61300be8bddf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_unseen_movies['predicted_rating'] = user_unseen_movies['movie_id'].apply(lambda x: svd.predict(user_id, x).est)


In [59]:
print(user_recommendations)

     movie_id                              title release_date  \
55         56                Pulp Fiction (1994)  01-Jan-1994   
479       480          North by Northwest (1959)  01-Jan-1959   
133       134                Citizen Kane (1941)  01-Jan-1941   
487       488                Sunset Blvd. (1950)  01-Jan-1950   
95         96  Terminator 2: Judgment Day (1991)  01-Jan-1991   

     video_release_date                                           imdb_url  \
55                  NaN  http://us.imdb.com/M/title-exact?Pulp%20Fictio...   
479                 NaN  http://us.imdb.com/M/title-exact?North%20by%20...   
133                 NaN  http://us.imdb.com/M/title-exact?Citizen%20Kan...   
487                 NaN  http://us.imdb.com/M/title-exact?Sunset%20Boul...   
95                  NaN  http://us.imdb.com/M/title-exact?Terminator%20...   

     predicted_rating  
55           4.335726  
479          4.222119  
133          4.221612  
487          4.218640  
95           4.20580

## **3. Kesimpulan**
Dalam proyek ini, kami berhasil membangun sistem rekomendasi menggunakan dataset MovieLens dan algoritma Collaborative Filtering dengan metode Matrix Factorization (SVD).

Sistem rekomendasi ini dapat memberikan rekomendasi film kepada pengguna berdasarkan preferensi mereka.

Proyek ini menunjukkan pentingnya penggunaan algoritma rekomendasi untuk memberikan pengalaman yang lebih personal kepada pengguna.

Kita dapat mengembangkan proyek ini lebih lanjut dengan mengeksplorasi teknik lain seperti Content-Based Filtering atau Hybrid Filtering untuk meningkatkan kinerja sistem rekomendasi.